Here I will calibrate the parameters given by the pictures provided

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import PIL.ExifTags
import PIL.Image
import pprint
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False, step = 100): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.yticks(np.arange(0, len(image), step))
    plt.xticks(np.arange(0, len(image[0]), step), rotation=90)
    plt.show() 

Ahora definimos el tamaño del tablero (la cantidad de puntos es igual a la cantidad de puntos en los que un cuadrado negro toca a uno blanco), y la lista de puntos a reconocer, tomando como origen (0, 0, 0) la esquina que se encuentra en las coordenadas (100, 100).

In [ ]:
chessBoardSize  = (8, 6)
objp = np.zeros((np.prod(chessBoardSize), 3),  dtype=np.float32)
objp[:, :2] = np.mgrid[0:chessBoardSize[0], 0:chessBoardSize[1]].T.reshape(-1, 2)
objp = objp * 28

Cargamos imagenes y dibujamos

In [ ]:
img_fnames = glob('./imagenes/img_cal_set1/*')
imgsGray = []
imgsColor = []
for imgName in img_fnames:
    img = cv.imread(imgName)
    imgsColor.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    imgsGray.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))


In [ ]:
def findCorners(imgsColor, imgsGray, plot=True, maxCount = 30, epsilon = 0.001, flag=cv.CALIB_CB_ADAPTIVE_THRESH):
    imgPoints = []
    objPoints = []
    criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_MAX_ITER, maxCount, epsilon)
    cb_flags = flag 
    for imgColor, imgGray in zip(imgsColor, imgsGray):
        imgColor = imgColor.copy()
        ret, corners = cv.findChessboardCorners(imgGray, chessBoardSize, flags=cb_flags)
        if ret:
            objPoints.append(objp)
            corners_subp = cv.cornerSubPix(imgGray, corners, (5, 5), (-1, -1), criteria)
            imgPoints.append(corners_subp)
            cv.drawChessboardCorners(imgColor, chessBoardSize, corners_subp, ret)
            if plot:
                plotter(imgColor)
    return imgPoints, objPoints

def calibrateCamera(objPoints, imgPoints, widthAndHeight, returnMatrix=False):
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objPoints, imgPoints, widthAndHeight, None, None, flags = cv.CALIB_ZERO_TANGENT_DIST)
    print('Camera Matrix: \n{}'.format(mtx))
    print('\nDistortion Coefficients: \n{}\n'.format(dist))
    if returnMatrix:
        return mtx, dist

def getIntrinsecParams(imgsColor, imgsGray):
    imgPoints, objPoints = findCorners(imgsColor, imgsGray, False)
    height, width = imgsGray[0].shape
    return calibrateCamera(objPoints, imgPoints, (width,height), returnMatrix=True)

In [ ]:
mtx, dist = getIntrinsecParams(imgsColor, imgsGray)
mtx1, dist1 = mtx, dist

In [ ]:
#Set 2
objp = np.zeros((np.prod(chessBoardSize), 3),  dtype=np.float32)
objp[:, :2] = np.mgrid[0:chessBoardSize[0], 0:chessBoardSize[1]].T.reshape(-1, 2)
objp = objp * 28
img_fnames = glob('./imagenes/img_cal_set2/*')
imgsGray = []
imgsColor = []
for imgName in img_fnames:
    img = cv.imread(imgName)
    imgsColor.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    imgsGray.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
mtx, dist = getIntrinsecParams(imgsColor, imgsGray)

Se ve que se tiene que usar el set 1 en vez del set dos porque el fx y el fy del set 1 son mas similares entre si, lo que implica que es mejor. Fuente: Arial 12

In [ ]:
#Levantamos la imagen para calibracion de parametros extrinsecos
objp = np.zeros((np.prod(chessBoardSize), 3),  dtype=np.float32)
objp[:, :2] = np.mgrid[0:chessBoardSize[0], 0:chessBoardSize[1]].T.reshape(-1, 2)
objp = objp * 28
img_fnames = glob('./imagenes/img_bloques/imgCalExtr.png')
imgsGray = []
imgsColor = []
for imgName in img_fnames:
    img = cv.imread(imgName)
    imgsColor.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    imgsGray.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
imgPoints, objPoints = findCorners(imgsColor, imgsGray, False)
mtx, dist = mtx1, dist1

In [ ]:
ret, rvecs, t = cv.solvePnP(objPoints[0], imgPoints[0], mtx, dist, useExtrinsicGuess=False)
rotation = cv.Rodrigues(rvecs)[0]
print(t)
print(rotation)

In [ ]:
# lo que hizo el chabon
def squareDif(p1, p2):
    return np.sqrt((p1[0]-p2[0]) ** 2 + (p1[1]-p2[1]) ** 2)
def processImage(img, i):
    areas = []
    centers = []
    dims = []
    rots = []
    img = cv.medianBlur(img, 5)
    imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, img_bin = cv.threshold(imgGray, 30, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    
    kernel = np.ones((2,2), np.uint8)
    opening = cv.morphologyEx(img_bin, cv.MORPH_OPEN, kernel)
    
    kernel = np.ones((2,2), np.uint8)
    ending = cv.morphologyEx(opening, cv.MORPH_CLOSE, kernel)
    
    contours, hierarchy = cv.findContours(ending, cv.RETR_TREE, cv.CHAIN_APPROX_TC89_L1)
    
    for cnt in contours:
        rect = cv.minAreaRect(cnt)
        center = rect[0]
        dim = rect[1]
        rot = rect[2]
        area = cv.contourArea(cnt)
        if (dim[0] * dim[1] != 0) and (area / (dim[0] * dim[1]) > 0.95) and area > 7000:
            box = cv.boxPoints(rect)
            po1 = box[0]
            po2 = box[1]
            po3 = box[3]
            if (squareDif(po3, po1) < squareDif(po2, po1)):
                rot = rot -90
            rot = abs(rot)
            print("encontre bloque")
            cv.circle(img, (int(center[0]), int(center[1])), 10, (0,0,255), 2)
            box = np.int0(box)
            cv.drawContours(img, [box], 0, (0,0,255), 2)
            plt.figure(i)
            plt.imshow(img)
            plt.show()
            print(f"area for image: {area}, rot: {rot}, dim: {dim}, center at: {center}")
            areas.append(area)
            centers.append(center)
            dims.append(dim)
            rots.append(rot)
    return areas, centers, dims, rots, box
        
            

In [ ]:
imgNames = glob('./imagenes/img_bloques/imgBloque*')
imgs = {}
for i in range(len(imgNames)):
    img = cv.imread(imgNames[i])
    imgs[i] = processImage(img, i)

In [ ]:
#Flasheada mia
#pruebo con solo una imagen
img = cv.imread(imgNames[0])
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
imgGray = cv.imread(imgNames[0], 0)
def createMask(img, samples, lowerMultiplier = 15, upperMultiplier = 6):
    meanColors = 0
    stdColors = 0
    for point1, point2, blockWidth in samples:
        colorMean, colorStd = cv.meanStdDev(img[point1[1]:point1[1] + blockWidth,point1[0]:point2[0], :])
        meanColors += colorMean
        stdColors += colorStd
    meanColors /= len(samples)
    stdColors /= len(samples)
    return cv.inRange(img, meanColors - stdColors * lowerMultiplier,  meanColors + stdColors * upperMultiplier)

In [ ]:
plotter(img, step=10)

In [ ]:
imgPrueba = img.copy()
block = ((370,270), (420, 270), 50)
cv.line(imgPrueba, block[0], block[1], (255,0,0), 5)

plotter(imgPrueba, step = 100)

In [ ]:
mask = createMask(img, [block], 3)
imgWithMask = cv.bitwise_and(img, img, mask=mask)
plotter(imgWithMask)
imgWithMaskGray = cv.cvtColor(imgWithMask, cv.COLOR_RGB2GRAY)
imgWithMaskGray = cv.medianBlur(imgWithMaskGray, 5)
plotter(imgWithMaskGray,grayScale=True)
imgGray = cv.medianBlur(imgGray, 5)
plotter(imgGray, grayScale=True)

In [ ]:
IMGS = [imgGray, imgWithMaskGray] #Estas son las imagenes que vamos a analizar
NAMES = ['Original', 'With Mask']
def otsuBinarization(imgs, thresh = 127):
    result = []
    for imgNum, img in enumerate(imgs):
        ret, imgBin = cv.threshold(img,thresh,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
        plotter(imgBin,NAMES[imgNum], grayScale=True)
        result.append(imgBin)
    return result

In [ ]:
imgBin, imgMaskBin = otsuBinarization([imgGray, imgWithMaskGray])

In [ ]:
# TESTING
testing = imgMaskBin.copy()
#kernelErode = np.ones((10,1), np.uint8)
#testing = cv.erode(testing, kernelErode, iterations = 5)
kernelDilate = np.ones((5,5), np.uint8)
testing = cv.erode(testing, kernelDilate, iterations=2)
plotter(testing, 'Dilate Modified', grayScale=True)
plotter(imgMaskBin)
imgDilated = imgGray.copy()#cv.erode(imgMaskBin.copy(), kernelDilate, iterations=2)
contours, hier = cv.findContours(imgDilated, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
realContours = []
out = img.copy()
for cnt in contours:
        rect = cv.minAreaRect(cnt)
        center = rect[0]
        dim = rect[1]
        rot = rect[2]
        area = cv.contourArea(cnt)
        if (dim[0] * dim[1] != 0) and (area / (dim[0] * dim[1]) > 0.95) and area > 7000:
            box = cv.boxPoints(rect)
            po1 = box[0]
            po2 = box[1]
            po3 = box[3]
            if (squareDif(po3, po1) < squareDif(po2, po1)):
                rot = rot -90
            rot = abs(rot)
            print("encontre bloque")
            cv.circle(img, (int(center[0]), int(center[1])), 10, (0,0,255), 2)
            box = np.int0(box)
            cv.drawContours(img, [box], 0, (0,0,255), 2)
            plt.figure(i)
            plt.imshow(img)
            plt.show()
            print(f"area for image: {area}, rot: {rot}, dim: {dim}, center at: {center}")
            realContours.append(cnt)
        
plotter(out)

In [ ]:
def getAspectRatio(contour):
    x,y,width,height = cv.boundingRect(contour)
    return np.round(float(width) / height, 2)

def getCentroid(moment, area):
    try:
        return (int(moment['m10'] / area), int(moment['m01'] / area))
    except:
        return 0

def getOrientation(contour):
    (x, y), (MA, mA), angle = cv.fitEllipse(contour)
    return (int(x), int(y)), (int(MA), int(mA)), np.round(angle)

In [ ]:
# Para mas info https://en.wikipedia.org/wiki/Image_moment
AREA = 0 #Para saber en que posicion del array que guarda el diccionario se encuentra la info deseada
PERIMETER = 1
CENTER = 2
ASPECT_RELATION = 3
ORIENTATION = 4
CONTOUR = 5

def getContoursInfo(contours):
    contoursArea = []
    contoursPerimeter = []
    contoursCenter = []

    for ctr in contours:
        moment = cv.moments(ctr)
        area = int(moment['m00'])
        contoursArea.append(area)
        contoursPerimeter.append(int(cv.arcLength(ctr,True)))
        contoursCenter.append(getCentroid(moment, area))

    contoursAreaMean = np.mean(contoursArea)

    contoursInfo = {}  
    contourNum = 1
    for area, perimeter, center, contour in  zip(contoursArea, contoursPerimeter, contoursCenter, contours):
        if area >= contoursAreaMean:
            contoursInfo[contourNum] = [area, perimeter, center, getAspectRatio(contour),
                                        getOrientation(contour), contour]
            contourNum += 1
            
    return contoursInfo

In [ ]:
contoursInfo = getContoursInfo(realContours)

def printResults(contoursInfo):
    print('RESULTS')
    for key, value in contoursInfo.items():
        print('''
                Block Num: {}

                    Area: {}
                    Perimeter: {}
                    Centroid: {}
                    Aspect Relation: {}
                    Orientation: {}
                '''.format(key, value[AREA], value[PERIMETER], value[CENTER], value[ASPECT_RELATION],
                          value[ORIENTATION][2]))
    

In [ ]:
plotter(img)

In [ ]:
printResults(contoursInfo)
centers = [x[CENTER] for x in contoursInfo.values()]

In [ ]:
mtx, dist, rotation, t

In [ ]:
type(mtx)
c = np.append(np.asarray(centers[0]), 1)
c

In [ ]:
INDEP = np.asarray([[1, 0,0,0], [0,1,0,0], [0,0,1,0]])


In [ ]:
a = []
for i in range(len(rotation)):
    a.append(np.append(rotation[i], t[i]))
a.append([0,0,0,1])
RT = np.asarray(a)

In [ ]:
RT

In [ ]:
def getCoord(center):
    # s * [uv1] = K * [R|T] *XwYwZw = k * (R XwYwZw + t)
    # ==> R^-1 * (K^-1 * [uv1] - t) = XwYwZw
    # K intrinsic params
    # R rotation
    # t translation
    # como s = R*XwYwZw[2] + t[2] para todo Zw, entonces => s =t[2]
    uv = np.array([[center[0],center[1],1]], dtype=np.float).T
    iK = np.linalg.inv(mtx)
    iR = np.linalg.inv(rotation)
    s = t[2]
    first = (s * iK.dot(uv) - t)
    return iR.dot(first)

In [ ]:
imgs

In [ ]:
def lichaPuto(v):
    v = [getCoord(x)[:2] for x in v]
    w = np.sqrt((v[1][0]-v[0][0])**2+(v[1][1]-v[0][1])**2)
    h = np.sqrt((v[3][0]-v[0][0])**2+(v[3][1]-v[0][1])**2)
    if w > h:
        w = np.sqrt((v[1][0]-v[3][0])**2+(v[1][1]-v[3][1])**2)
    else:
        h = np.sqrt((v[1][0]-v[3][0])**2+(v[1][1]-v[3][1])**2)
    if w > h:
        return (w, h)
    else:
        return (h, w)

In [ ]:
for v in imgs.values():
    l = lichaPuto(v[-1])
    print(f"area es: {l[0] * l[1]}, dimension: {l}")